In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession,Row
from pyspark.sql.window import Window
from pyspark.sql.functions import when, count, col, desc, asc, row_number, mean
from pyspark.sql.types import (StructField,FloatType,IntegerType,StructType)
from pyspark.ml.clustering import KMeans, KMeansModel
#from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
spark = SparkSession.builder.appName('iteration_4').getOrCreate()
import numpy as np
from numpy import array
from math import sqrt

In [3]:
df = spark.read.load("./datasets/covid19_impact.csv", format="csv", header="true")

In [4]:
df.show()

+----+-----------+----------+-----+---+---+---+----------+-----------+
|CODE|    COUNTRY|      DATE|  HDI| TC| TD|STI|       POP|     GDPCAP|
+----+-----------+----------+-----+---+---+---+----------+-----------+
| AFG|Afghanistan|31/12/2019|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|Afghanistan|01/01/2020|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|Afghanistan|02/01/2020|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|Afghanistan|03/01/2020|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|Afghanistan|04/01/2020|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|Afghanistan|05/01/2020|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|Afghanistan|06/01/2020|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|Afghanistan|07/01/2020|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|Afghanistan|08/01/2020|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|Afghanistan|09/01/2020|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|Afghanistan|10/01/2020|0.498|  0|  0|  0|17.4772331|7.497754494|
| AFG|

In [5]:
df.count()

50418

In [6]:
df.select("COUNTRY").distinct().count()

210

In [7]:
df.printSchema()

root
 |-- CODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- HDI: string (nullable = true)
 |-- TC: string (nullable = true)
 |-- TD: string (nullable = true)
 |-- STI: string (nullable = true)
 |-- POP: string (nullable = true)
 |-- GDPCAP: string (nullable = true)



In [8]:
dm = spark.read.load("./datasets/UNdata_Poverty.csv", format="csv", header="true")

In [9]:
dm.show()

+---------------+----+-----+---------------+
|Country or Area|Year|Value|Value Footnotes|
+---------------+----+-----+---------------+
|        Albania|2017|  8.2|              1|
|        Albania|2016|  9.4|              1|
|        Albania|2015|  8.2|              1|
|        Albania|2014| 11.6|              1|
|        Albania|2012|  6.2|              2|
|        Albania|2008|  5.3|              2|
|        Albania|2005|  8.7|              2|
|        Albania|2002| 14.1|              2|
|        Albania|1996| 11.2|              3|
|        Algeria|2011|  3.7|              4|
|        Algeria|1995| 23.5|              5|
|        Algeria|1988| 24.5|              6|
|         Angola|2018| 73.2|              7|
|         Angola|2008| 60.6|              8|
|         Angola|2000| 59.7|              1|
|      Argentina|2018|  3.9|              9|
|      Argentina|2017|  3.1|              9|
|      Argentina|2016|  3.4|              9|
|      Argentina|2014|  3.5|              9|
|      Arg

In [10]:
dm.count()

2146

In [11]:
dm.select("Country or Area").distinct().count()

179

In [12]:
dm.printSchema()

root
 |-- Country or Area: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Value Footnotes: string (nullable = true)



In [13]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+-------+----+----+---+---+---+---+------+
|CODE|COUNTRY|DATE| HDI| TC| TD|STI|POP|GDPCAP|
+----+-------+----+----+---+---+---+---+------+
|   0|      0|   0|6202|  0|  0|  0|  0|     0|
+----+-------+----+----+---+---+---+---+------+



In [14]:
dm.select([count(when(col(c).isNull(), c)).alias(c) for c in dm.columns]).show()

+---------------+----+-----+---------------+
|Country or Area|Year|Value|Value Footnotes|
+---------------+----+-----+---------------+
|              0|   0|   47|            209|
+---------------+----+-----+---------------+



In [15]:
df.columns

['CODE', 'COUNTRY', 'DATE', 'HDI', 'TC', 'TD', 'STI', 'POP', 'GDPCAP']

In [16]:
# Selecting items
w = Window.partitionBy("Country or Area").orderBy(col("Year").desc())
dm2 = dm.withColumn("row",row_number().over(w)) \
      .filter(col("row") == 1).drop("row") \
      .orderBy(col("Country or Area").asc())
dm2.show()

+--------------------+----+-----+---------------+
|     Country or Area|Year|Value|Value Footnotes|
+--------------------+----+-----+---------------+
|             Albania|2017|  8.2|              1|
|             Algeria|2011|  3.7|              4|
|              Angola|2018| 73.2|              7|
|           Argentina|2018|  3.9|              9|
|             Armenia|2018|  9.4|             12|
|           Australia|2014|  0.7|             13|
|             Austria|2017|  0.4|             17|
|          Azerbaijan|2005|    0|              1|
|          Bangladesh|2016| 52.5|             21|
|             Belarus|2018|    0|             24|
|             Belgium|2017|  0.2|             17|
|              Belize|1999| 27.7|             27|
|               Benin|2015| 76.3|             29|
|              Bhutan|2017| 12.2|             31|
|             Bolivia|2018| 10.5|             32|
|Bosnia and Herzeg...|2011|  0.7|              1|
|            Botswana|2015| 36.5|             37|


In [17]:
# Selecting items
w2 = Window.partitionBy("COUNTRY").orderBy(col("TC").desc())
df2 = df.withColumn("row",row_number().over(w2)) \
      .filter(col("row") == 1).drop("row") \
      .orderBy(col("COUNTRY").asc())
df2.show()

+----+-------------------+----------+-----+-----------+-----------+-----------+-----------+-----------+
|CODE|            COUNTRY|      DATE|  HDI|         TC|         TD|        STI|        POP|     GDPCAP|
+----+-------------------+----------+-----+-----------+-----------+-----------+-----------+-----------+
| AFG|        Afghanistan|10/06/2020|0.498|9.973899417|5.950642553|4.365643155| 17.4772331|7.497754494|
| ALB|            Albania|19/10/2020|0.785|9.744198695| 6.11146734|          0|14.87253667|9.376145531|
| DZA|            Algeria|18/07/2020|0.754|9.996431298|6.963189986|4.305010592|17.59630906|9.540639235|
| AND|            Andorra|17/10/2020|0.858| 8.12474302|4.077537444|          0|11.25499635|          0|
| AGO|             Angola|19/10/2020|0.581|8.917578754|5.484796933|          0| 17.3079574|8.668968767|
| AIA|           Anguilla|03/04/2020| null|1.098612289|          0|4.305010592|9.615938805|          0|
| ATG|Antigua and Barbuda|18/10/2020| 0.78|4.779123493|1.0986122

In [18]:
# drop columns
df3 = df2.drop("CODE", "STI")
dm3 = dm2.drop("Value Footnotes")
df3.printSchema()
dm3.printSchema()
df3.show()
dm3.show()

root
 |-- COUNTRY: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- HDI: string (nullable = true)
 |-- TC: string (nullable = true)
 |-- TD: string (nullable = true)
 |-- POP: string (nullable = true)
 |-- GDPCAP: string (nullable = true)

root
 |-- Country or Area: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Value: string (nullable = true)

+-------------------+----------+-----+-----------+-----------+-----------+-----------+
|            COUNTRY|      DATE|  HDI|         TC|         TD|        POP|     GDPCAP|
+-------------------+----------+-----+-----------+-----------+-----------+-----------+
|        Afghanistan|10/06/2020|0.498|9.973899417|5.950642553| 17.4772331|7.497754494|
|            Albania|19/10/2020|0.785|9.744198695| 6.11146734|14.87253667|9.376145531|
|            Algeria|18/07/2020|0.754|9.996431298|6.963189986|17.59630906|9.540639235|
|            Andorra|17/10/2020|0.858| 8.12474302|4.077537444|11.25499635|          0|
|       

In [19]:
# checking null and mean value
df3.select([count(when(col(c).isNull(), c)).alias(c) for c in df3.columns]).show()
dm3.select([count(when(col(c).isNull(), c)).alias(c) for c in dm3.columns]).show()
mean_hdi = df3.select(mean(df3['HDI'])).collect()
mean_hdi_val = mean_hdi[0][0]
print(mean_hdi_val)



+-------+----+---+---+---+---+------+
|COUNTRY|DATE|HDI| TC| TD|POP|GDPCAP|
+-------+----+---+---+---+---+------+
|      0|   0| 28|  0|  0|  0|     0|
+-------+----+---+---+---+---+------+

+---------------+----+-----+
|Country or Area|Year|Value|
+---------------+----+-----+
|              0|   0|    0|
+---------------+----+-----+

0.7064175824175828


In [20]:
# replacing null
df4 = df3.na.fill('0.7064', subset=['HDI'])
df4.select([count(when(col(c).isNull(), c)).alias(c) for c in df4.columns]).show()
df5 = df4.drop("DATE")

+-------+----+---+---+---+---+------+
|COUNTRY|DATE|HDI| TC| TD|POP|GDPCAP|
+-------+----+---+---+---+---+------+
|      0|   0|  0|  0|  0|  0|     0|
+-------+----+---+---+---+---+------+



In [21]:
# data casting
df_cast = (df5.withColumn("HDI", df5["HDI"].cast("float")) \
       .withColumn("TC", df5["TC"].cast("float")) \
       .withColumn("TD", df5["TD"].cast("float")) \
       .withColumn("POP", df5["POP"].cast("float")) \
       .withColumn("GDPCAP", df5["GDPCAP"].cast("float")))
dm_cast = (dm3.withColumn("Year", dm3["Year"].cast(IntegerType())) \
           .withColumn("Value", dm3["Value"].cast("float")))
df_cast.printSchema()
dm_cast.printSchema()

root
 |-- COUNTRY: string (nullable = true)
 |-- HDI: float (nullable = true)
 |-- TC: float (nullable = true)
 |-- TD: float (nullable = true)
 |-- POP: float (nullable = true)
 |-- GDPCAP: float (nullable = true)

root
 |-- Country or Area: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Value: float (nullable = true)



In [22]:
# renaming
df6 = df_cast.withColumn("GDP", df_cast.GDPCAP*df_cast.POP)
df7 = (df6.withColumnRenamed('HDI','Human_Development_Index') \
       .withColumnRenamed('TC', 'Total_Case') \
       .withColumnRenamed('TD', 'Total_Death') \
       .withColumnRenamed('POP', 'Population') \
       .withColumnRenamed('GDPCAP', 'GDP/Capita'))
df7.show()
df7.printSchema()
dm4 = dm_cast.withColumnRenamed('Value','Poverty_Rate_(%)')

+-------------------+-----------------------+----------+-----------+----------+----------+----------+
|            COUNTRY|Human_Development_Index|Total_Case|Total_Death|Population|GDP/Capita|       GDP|
+-------------------+-----------------------+----------+-----------+----------+----------+----------+
|        Afghanistan|                  0.498|    9.9739|  5.9506426| 17.477234| 7.4977546| 131.04001|
|            Albania|                  0.785|  9.744199|  6.1114674| 14.872537|  9.376145| 139.44707|
|            Algeria|                  0.754|  9.996431|    6.96319|  17.59631|  9.540639| 167.88004|
|            Andorra|                  0.858|  8.124743|  4.0775375| 11.254996|       0.0|       0.0|
|             Angola|                  0.581|  8.917579|   5.484797| 17.307957|  8.668969| 150.04214|
|           Anguilla|                 0.7064| 1.0986123|        0.0|  9.615939|       0.0|       0.0|
|Antigua and Barbuda|                   0.78| 4.7791233|  1.0986123| 11.491988|  9

In [23]:
# Integrate various data sources
merged_data = df7.join(dm4,df7["COUNTRY"] == dm4["Country or Area"])
merged_data = merged_data.drop("Country or Area")
merged_data.show(truncate=8)
merged_data.count()

+--------+-----------------------+----------+-----------+----------+----------+--------+----+----------------+
| COUNTRY|Human_Development_Index|Total_Case|Total_Death|Population|GDP/Capita|     GDP|Year|Poverty_Rate_(%)|
+--------+-----------------------+----------+-----------+----------+----------+--------+----+----------------+
| Albania|                  0.785|  9.744199|   6.111...|  14.87...|  9.376145|139.4...|2017|             8.2|
| Algeria|                  0.754|  9.996431|    6.96319|  17.59631|  9.540639|167.8...|2011|             3.7|
|  Angola|                  0.581|  8.917579|   5.484797|  17.30...|  8.668969|150.0...|2018|            73.2|
|Argen...|                  0.825|  9.999116|   6.473891|  17.62...|   9.84871|173.5...|2018|             3.9|
| Armenia|                  0.755|   9.98585|   5.955...|  14.90...|  9.081095|135.3...|2018|             9.4|
|Austr...|                  0.939|  9.985666|   5.802...|  17.05...|  10.70...|182.5...|2014|             0.7|
|

143

In [24]:
# ordering merged dataset
merged_data1 = merged_data.orderBy(col("Poverty_Rate_(%)").desc())
merged_data1.show(truncate=8)

+--------+-----------------------+----------+-----------+----------+----------+--------+----+----------------+
| COUNTRY|Human_Development_Index|Total_Case|Total_Death|Population|GDP/Capita|     GDP|Year|Poverty_Rate_(%)|
+--------+-----------------------+----------+-----------+----------+----------+--------+----+----------------+
|Madag...|                  0.519|   9.72973|   5.472...|  17.13662|  7.255902|124.3...|2012|            90.9|
| Burundi|                  0.417|  6.295266|        0.0|  16.29...|  6.554254|106.7...|2013|            89.6|
|  Malawi|                  0.477|  8.675393|   5.198497|  16.76...|  6.998548|117.3...|2016|            89.6|
|Uzbek...|                   0.71|  9.976088|   4.804021|  17.32...|  8.740833|151.4...|2003|            86.2|
|Guine...|                  0.455|  7.778...|   3.713572|  14.49...|  7.345...|106.4...|2010|            85.4|
|Centr...|                  0.367|  8.487764|   4.127...|  15.39...|  6.494117|99.94646|2008|            82.9|
|

In [25]:
panda = merged_data1.toPandas()
pearson_col = panda.corr('pearson')
pearson_col

,Human_Development_Index,Total_Case,Total_Death,Population,GDP/Capita,GDP,Year,Poverty_Rate_(%)
Human_Development_Index,1.000000,0.335720,0.275682,-0.039802,0.904708,0.694713,0.204135,-0.792687
Total_Case,0.335720,1.000000,0.843036,0.453416,0.368158,0.585251,0.160745,-0.322620
Total_Death,0.275682,0.843036,1.000000,0.513503,0.317518,0.591467,0.190197,-0.282964
Population,-0.039802,0.453416,0.513503,1.000000,-0.071808,0.605519,0.206683,0.134236
GDP/Capita,0.904708,0.368158,0.317518,-0.071808,1.000000,0.747305,0.208286,-0.860788
GDP,0.694713,0.585251,0.591467,0.605519,0.747305,1.000000,0.300350,-0.603658
Year,0.204135,0.160745,0.190197,0.206683,0.208286,0.300350,1.000000,-0.279900
Poverty_Rate_(%),-0.792687,-0.322620,-0.282964,0.134236,-0.860788,-0.603658,-0.279900,1.000000


In [26]:
dfm = merged_data1.drop("COUNTRY", "Year", "Population")
dfm.show()

+-----------------------+----------+-----------+----------+----------+----------------+
|Human_Development_Index|Total_Case|Total_Death|GDP/Capita|       GDP|Poverty_Rate_(%)|
+-----------------------+----------+-----------+----------+----------+----------------+
|                  0.519|   9.72973|  5.4722705|  7.255902| 124.34163|            90.9|
|                  0.417|  6.295266|        0.0|  6.554254|106.777145|            89.6|
|                  0.477|  8.675393|   5.198497|  6.998548| 117.34302|            89.6|
|                   0.71|  9.976088|   4.804021|  8.740833| 151.44487|            86.2|
|                  0.455| 7.7786303|   3.713572| 7.3451552| 106.44986|            85.4|
|                  0.367|  8.487764|  4.1271343|  6.494117|  99.94646|            82.9|
|                  0.437|  9.293394|   4.317488| 7.0353594| 121.41415|            82.4|
|                  0.524|   8.51198|  3.5263605| 7.5252147| 123.23877|            80.3|
|                  0.427|  8.125

In [27]:
assembler = VectorAssembler(inputCols= dfm.columns, outputCol='features')
df_final = assembler.transform(dfm)

In [28]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True)
scaledModel = scaler.fit(df_final)
df_final = scaledModel.transform(df_final)
#df_final.show()

In [31]:
kmeans = KMeans(featuresCol='scaledFeatures', k=4)
model = kmeans.fit(df_final)
prediction = model.transform(df_final)


eval = Evaluator()
silihouette = eval.evaluate(prediction)

parsedData = dfm.map(lambda line: array([float(x) for x in line.split(' ')]))
clusters = KMeans.train(parsedData, 4, maxIterations=10, initializationMode="random")

df3.write.format("csv").option("header",True).mode("overwrite").save("./datasets/covid19_output.csv")

covid_output = spark.read.load("./datasets/covid19_output.csv", format="csv", header="true")
covid_output.show()